# Fingerprint correlation reproduction
## Install external libraries

In [ ]:
!pip install torch torchvision
!pip install transformers
!pip install rdkit
!pip install selfies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import libraries

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy.spatial.distance import cdist, cosine
from scipy.stats import pearsonr, spearmanr
from rdkit import Chem

import logging
import random
from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd
import selfies

from transformers import AutoModelForMaskedLM, AutoTokenizer

SEED = 6217

## Sampling functions

In [ ]:
def canonize_smile(input_str: str, remove_identities: bool = True) -> str:
    mol = Chem.MolFromSmiles(input_str)
    if mol is None:
        return None
    if remove_identities:
        [a.SetAtomMapNum(0) for a in mol.GetAtoms()]
    return Chem.MolToSmiles(mol)

### Random sampling

In [ ]:
def sample_synonym(
    seed: int = None, min_size: int = 5, max_size: int = 30
) -> Tuple[str, str, str, str]:
    random.seed(seed)
    found_fitting = False
    alphabet = (
        selfies.get_semantic_robust_alphabet()
    )  # Gets the alphabet of robust symbols
    while not found_fitting:
        size = random.randint(min_size, max_size)
        rnd_selfies = "".join(random.sample(list(alphabet), size))
        rnd_smiles = selfies.decoder(rnd_selfies)
        rnd_smiles = rnd_smiles.replace("-1", "-").replace("+1", "+")
        canon_smiles = canonize_smile(rnd_smiles)
        rnd_selfies = selfies.encoder(rnd_smiles)
        canon_selfies = selfies.encoder(canon_smiles)
        if (canon_smiles != rnd_smiles) and (canon_selfies != rnd_selfies):
            found_fitting = True
    return rnd_smiles, canon_smiles, rnd_selfies, canon_selfies


def sample_random_molecules(
    amount: int = 1000, overcompensation_factor: int = 1.1
) -> pd.DataFrame:
    molecule_list = []
    for seed in range(SEED, int(SEED + amount * overcompensation_factor)):
        molecule_list.append(sample_synonym(seed))
    df = pd.DataFrame(
        molecule_list,
        columns=["rnd_smiles", "canon_smiles", "rnd_selfies", "canon_selfies"],
    )
    df.drop_duplicates(["canon_smiles"], inplace=True)
    if df.shape[0] < amount:
        logging.info(
            f"Overcompensation factor of {overcompensation_factor} was not enough."
        )
        return sample_random_molecules(amount, overcompensation_factor * 1.1)
    return df.iloc[:amount]

### PubChem sampling

In [ ]:
def sample_mols_fromPubChem():
    # dataset taken from https://ibm.ent.box.com/v/MoLFormer-data ("pubchem-smiles-canonical.zip")
    smi_df = pd.read_csv(
        "/content/drive/MyDrive/Datasets/smi_test.smi", header=None, delimiter=r"\s+"
    )
    return smi_df.iloc[:1000]

## Similarity computation functions

In [ ]:
def get_correlation(distances1, distances2, prefix: Optional[str] = None):
    pearson = pearsonr(distances1, distances2)
    spearman = spearmanr(distances1, distances2)
    pearson_string = f"{prefix} Pearson" if prefix else "Pearson"
    spearman_string = f"{prefix} Spearman" if prefix else "Spearman"
    return {pearson_string: pearson, spearman_string: spearman}

### Chemical fingerprint similarity

In [ ]:
def get_pairwise_chemical_similarities(mol_start, mol_end):
    # https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints
    morgan_start = AllChem.GetMorganFingerprint(mol_start, 2)  # 2 is closest to ECFP
    morgan_end = AllChem.GetMorganFingerprint(mol_end, 2)  # 2 is closest to ECFP
    morgan_dice = DataStructs.DiceSimilarity(morgan_start, morgan_end)
    morgan_start = AllChem.GetMorganFingerprintAsBitVect(mol_start, 2)
    morgan_end = AllChem.GetMorganFingerprintAsBitVect(mol_end, 2)
    morgan_tanimoto = DataStructs.FingerprintSimilarity(morgan_start, morgan_end)
    rdkit_start = Chem.RDKFingerprint(mol_start)
    rdkit_end = Chem.RDKFingerprint(mol_end)
    rdkit_distance = DataStructs.FingerprintSimilarity(rdkit_start, rdkit_end)
    return morgan_dice, morgan_tanimoto, rdkit_distance


def get_chemical_similarities(SMILES_starts, SMILES_ends):
    mol_starts = [Chem.MolFromSmiles(SMILES_start) for SMILES_start in SMILES_starts]
    mol_ends = [Chem.MolFromSmiles(SMILES_end) for SMILES_end in SMILES_ends]
    similarities = np.array(
        [
            get_pairwise_chemical_similarities(mol_start, mol_end)
            for (mol_start, mol_end) in zip(mol_starts, mol_ends)
        ]
    )
    return similarities

### Model embedding distance

In [ ]:
def compute_distances(start: pd.Series, end: pd.Series) -> Tuple[float, float, float]:
    euclid = cdist([start], [end], "euclid")
    manhattan = cdist([start], [end], "cityblock")
    cos = cosine(start, end)
    return euclid[0][0], manhattan[0][0], cos


def get_embeddings(tokenizer, SMILES_starts_nn, SMILES_ends_nn):
    SMILES_starts = np.array(SMILES_starts_nn)
    SMILES_ends = np.array(SMILES_ends_nn)
    mol_starts_tokenized = [
        tokenizer.encode(SMILES_start, return_tensors="pt")
        for SMILES_start in SMILES_starts
    ]
    mol_ends_tokenized = [
        tokenizer.encode(SMILES_end, return_tensors="pt") for SMILES_end in SMILES_ends
    ]
    # https://stackoverflow.com/questions/63323464/how-to-get-the-correct-embedding-from-roberta-transformers
    mol_starts_out = [model(input)[-1][-1] for input in mol_starts_tokenized]
    mol_ends_out = [model(input)[-1][-1] for input in mol_ends_tokenized]
    return mol_starts_out, mol_ends_out

# Main

## Mount Google Drive

In [ ]:
#read in data from file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Sample molecules

In [ ]:
# sample random mols without file
rndm_mols = sample_random_molecules(1000)
rndm_smiles = rndm_mols.loc[:,"rnd_smiles"]
# sample mols from PubChem
#rndm_mols = sample_mols_fromPubChem()
#rndm_smiles = rndm_mols.loc[:1000, 1]

# Only sample 300 molecules because of Colab size restrictions
smiles_starts = rndm_smiles.head(300)
smiles_ends = rndm_smiles.tail(300)
print(smiles_starts.head(5))

# Else to use all 1000/2 molecules
# top = int(np.ceil(rndm_smiles.shape[0] / 2))
# bottom = int(rndm_smiles.shape[0] - top)
# smiles_starts = rndm_smiles.head(top)
# smiles_ends = rndm_smiles.tail(bottom)

[15:35:29] WARNING: not removing hydrogen atom without neighbors
[15:35:30] WARNING: not removing hydrogen atom without neighbors


0    [P+]=[C-]C=CN=[B+]
1            [O-]S=[B+]
2                 S#COI
3                 [H]Br
4                   O=N
Name: rnd_smiles, dtype: object


## Get embeddings

In [ ]:
# Testing different models
#    model 1
#model = AutoModelForMaskedLM.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
#tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
#    model2 : ChemBERTa_zinc250k_v2_40k supposedly better
#model = AutoModelForMaskedLM.from_pretrained("seyonec/ChemBERTa_zinc250k_v2_40k")
#tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa_zinc250k_v2_40k")
#    model 3 biggest and newest: "DeepChem/ChemBERTa-77M-MLM"
model = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")

# get embeddings of SMILES
smiles_starts_embs, smiles_ends_embs = get_embeddings(
    tokenizer, smiles_starts, smiles_ends
)
print(len(smiles_starts_embs))
# turn to numpy arrays
smiles_starts_embs = [em.detach().numpy() for em in smiles_starts_embs]
smiles_ends_embs = [em.detach().numpy() for em in smiles_ends_embs]
# print(f"First molecule embedding: {smiles_starts_embs[0]}"")

300


## Compute distances and similarities

In [ ]:
# get chemical similarities of SMILES
chemical_similarities = get_chemical_similarities(smiles_starts, smiles_ends)
morgan_dice = chemical_similarities[:, 0]
morgan_tanimoto = chemical_similarities[:, 1]
rdkit_distance = chemical_similarities[:, 2]

print(len(smiles_starts_embs))
print(len(smiles_starts_embs[0][-1]))

# compute pairwise distances of embeddings
distances = [
    compute_distances(emb_start[-1], emb_end[-1])
    for (emb_start, emb_end) in zip(smiles_starts_embs, smiles_ends_embs)
]
print("Pairs tested: ", len(distances))

300
600
Pairs tested:  300


## Compute correlations

In [ ]:
distances_des = ["euclid", "manhattan", "cosine"]
sims = [morgan_dice, morgan_tanimoto, rdkit_distance]
sims_des = ["morgan_dice", "morgan_tanimoto", "rdkit_distance"]
for iti, sim in enumerate(sims):
    for pos, distance_de in enumerate(distances_des):
        print(f"correlation {sims_des[iti]} and {distance_de}")
        cor = get_correlation([distance[pos] for distance in distances], sim)
        print(cor)

correlation morgan_dice and euclid
{'Pearson': PearsonRResult(statistic=-0.19492488134786223, pvalue=0.0006869791654329817), 'Spearman': SignificanceResult(statistic=-0.17230598099496452, pvalue=0.0027498935040403245)}
correlation morgan_dice and manhattan
{'Pearson': PearsonRResult(statistic=-0.17186273233780103, pvalue=0.002821210615453053), 'Spearman': SignificanceResult(statistic=-0.15296802798017836, pvalue=0.007952953471339402)}
correlation morgan_dice and cosine
{'Pearson': PearsonRResult(statistic=-0.17786320031667996, pvalue=0.001984683383943188), 'Spearman': SignificanceResult(statistic=-0.25665405186281814, pvalue=6.716996455108489e-06)}
correlation morgan_tanimoto and euclid
{'Pearson': PearsonRResult(statistic=-0.1976050433435225, pvalue=0.0005767140560968044), 'Spearman': SignificanceResult(statistic=-0.17207334167933616, pvalue=0.0027871173966223625)}
correlation morgan_tanimoto and manhattan
{'Pearson': PearsonRResult(statistic=-0.18552710897375962, pvalue=0.00124624718